In [3]:
#Extract (from only Sloane records, no indexs) all catnum where place or person name (or both) mentioned with 
#page numbers<pb>, <div2>, and text within <div3>
#There are div3s with no catnum.
#Last modified Feb 2020
from lxml import etree
import re
import csv
import pandas as pd

In [4]:
path = '/Users/deborahleem/Documents/1_PhD/SloaneDBwork19/Miscellanies5Ah_MASTER_fixed_whitespaceDL.xml'
doc = etree.parse(open(path))
root = doc.getroot()

In [5]:
def remove_element(el):
    parent = el.getparent()
    if el.tail:
        prev = el.getprevious()
        if prev:
            prev.tail = (prev.tail or '') + el.tail
        else:
            parent.text = (parent.text or '') + el.tail
    parent.remove(el)

# Remove every <add rend="del"> and <add rend="pencil"> from document before the rest of analysis
for e in list(root.iterfind('.//{*}add')):
    if ('rend' in e.attrib) and (e.attrib['rend'] == 'del' or e.attrib['rend'] == 'pencil'):
        #print('removing', etree.tostring(e, pretty_print=True))
        if e.text == '-1861' or e.text == '2094.' or e.text == '-382.':
            continue
        remove_element(e)

/Users/deborahleem/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: The behavior of this method will change in future versions. Use specific 'len(elem)' or 'elem is not None' test instead.
  """


In [6]:
part_excluded = {
  "EPF108491027v": ["2108","2109","2110","2111"], # "Miscellanies"
  "EPF108491177v": ["1","2"], # "Antiquities"
  "EPF108491277": ["57"], # "Mathematical"
}

excluded = {
  "EPF108491178v": ["1","2","3","4"], # "Antiquities"
  "EPF108491179v": ["4","5","6","7","8","1"], # "Antiquities"
  "EPF108491180v": ["2","3"], # "Antiquities"
  "EPF108491181v": ["4","5"], # "Antiquities"
  "EPF108491182v": ["1","2","3","4","5"], # "Antiquities"
  "EPF108491183v": ["6","7","8","9","10","11","12"], # "Antiquities"
  "EPF108491184v": ["13","14","1","2","3","4","5","1"], # "Antiquities"
  "EPF108491186v": ["1","2","3","4","5","6","7","8","9","10","11","12","13","14","15","16","17","18","19"], # "Antiquities"
  "EPF108491187v": ["20","21","22","23","24","25","26","27","28","29","30","31","32","33","34","35","36","37","38","39"], # "Antiquities"
  "EPF108491188v": ["40","41","42","43","44","45","46","47","48","1","2","3","4","5","6","7","8","9"], # "Antiquities"
  "EPF108491189v": ["10","11","12","1","2","3","4","5","6"], # "Antiquities"
  "EPF108491190v": ["7","8","9","10","11"], # "Antiquities"
}
#last two entries on EPF108491271 wrapped in div3 are not Sloane, Pictures &c.

In [7]:
def find_div2_label(element):
    while element is not None:
        if element.tag == '{http://www.tei-c.org/ns/1.0}div2':
            label = element.find('{*}label')
            if label is not None:
                return ''.join(label.itertext()).strip()
            return
        element = element.getparent()

In [8]:
def find_enclosing_div3(element):
    while element is not None:
        if element.tag == '{http://www.tei-c.org/ns/1.0}div3':
            return element
        element = element.getparent()
    
def find_pb_xml_id(element):
    element = find_enclosing_div3(element)
    while element is not None:
        if element.tag == '{http://www.tei-c.org/ns/1.0}pb':
            return element.attrib['{http://www.w3.org/XML/1998/namespace}id']
        element = element.getprevious()

In [9]:
def cleanup_catnum(text):
    #removing from beginning and end:
    # \s spaces/tabs/newlines
    # \- dashes
    # . dots
    # ? question marks
    # _ underscores
    # ✓
    text = re.sub(r'^[ ✓\t\n.\-_?]+', "", text)
    text = re.sub(r'[ ✓\t\n.\-_?]+$', "", text)
    text = re.sub(r'[ ✓\t\n.\-_?]+\*', "*", text)
    text = text.replace('. ', ' ')
    return text

In [11]:
#ignore non-Sloane catnums and index <label type="index">. End of index </div2>
people_csv_data = []

# labels = set()

for div3 in root.iter("{*}div3"):
    persons = list(div3.iter('{*}persName'))
    places = list(div3.iter('{*}placeName'))
    catnums = list(div3.iter('{*}catnum'))
    clean_catnums = [cleanup_catnum(''.join(catnum.itertext())) for catnum in catnums]
    
    page_id = find_pb_xml_id(div3)
    label = find_div2_label(div3)
    
#     labels.add(label)
    if label.startswith('Index'):
        continue
    
    if len(persons) > 0 or len(places) > 0:
        if page_id in excluded:
            continue

#         This is an individual row to be saved as a dict
        res = {}

        res["div2_label"] = label
        div3_text = ''.join(div3.itertext())

        person_res = []
        for person in persons:
            name = re.sub(r'\s+', " ", ''.join(person.itertext()))
            name = re.sub(r'^\s+', '', name)
            name = re.sub(r'\s+$', '', name)  
            person_res.append(name)

        res["person"] = person_res

        places_res = []
        for place in places:
            name = re.sub(r'\s+', " ", ''.join(place.itertext()))
            name = re.sub(r'^\s+', '', name)
            name = re.sub(r'\s+$', '', name)   
            places_res.append(name)

        res["place"] = places_res

        catnum_res = []
        
#         Clean the catnums
        for name in clean_catnums:
            if not name:
                name = ''
            catnum_res.append(name)
            
        if not catnum_res:
            catnum_res = ['']
        res["catnum"] = catnum_res
        people_csv_data.append(res)

# print(labels)

In [12]:
# Create the dataframe and populate based on the dictionary

df = pd.DataFrame({'div2_label': [], 'person': [], 'place': [], 'catnum': []})
for row in people_csv_data:
    df = df.append({'div2_label': row['div2_label'], 'person': str(row['person']), 'place': str(row['place']), 'catnum': str(row['catnum'])}, ignore_index=True)
df.head()

,div2_label,person,place,catnum
0,Miscellanies.,['Mr. Amyand.'],['East Indies'],['1799']
1,Miscellanies.,['Mr. Theobalds.'],['Norway'],['1801']
2,Miscellanies.,['Mr. Roberts'],[],['1806']
3,Miscellanies.,['Mr. M'],['Lisbon'],['1807']
4,Miscellanies.,[],['Lisbone'],['1808']


In [13]:
# number of records
len(df)

1401

In [14]:
# records with no person. No of records with place name only.
df[df.person == '[]']
# len(df[df.person == '[]'])

,div2_label,person,place,catnum
4,Miscellanies.,[],['Lisbone'],['1808']
6,Miscellanies.,[],['China'],['1810']
10,Miscellanies.,[],['Malabar'],['1818']
11,Miscellanies.,[],['England'],['1819']
12,Miscellanies.,[],['London'],['1']
13,Miscellanies.,[],['Coromandel'],['4']
15,Miscellanies.,[],['Persian'],['8']
20,Miscellanies.,[],['Surinam'],['1823']
21,Miscellanies.,[],['Guinea'],['1830']
23,Miscellanies.,[],['Guinea'],['1833']


In [15]:
#total number of place name (count every place name element)


In [16]:
# records with no person or place
df[(df.person == '[]') & (df.place == "[]")]

,div2_label,person,place,catnum


In [17]:
# records with no place. Total no of records with person only.
len(df[df.place == "[]"])

555

In [18]:
# Both person and place are populated
len(df[(df.person != '[]') & (df.place != "[]")])

343

In [19]:
df[df.catnum == "['1823']"]

,div2_label,person,place,catnum
20,Miscellanies.,[],['Surinam'],['1823']


In [61]:
with open('people_and_places.csv', 'w') as file:
    writer = csv.writer(file)
    for row in people_csv_data:
        writer.writerow(row.values())

In [ ]:
#1. How many person names? Create a list of entries with person name. with div2 label, catnum, div3, and page number.
#Do the same with place name separately


In [ ]:
#3. How many with only place names? Create a list of these with their div2 label, catnum, div3, and page number.

In [ ]:
#4. List of both person and place name with div2 label, catnum, div3, and page number.

In [ ]:
#5. Identify if there is a most commonly occuring combination of person and place name.

In [22]:
def fn(place):
    place_list = eval(place)
    return len(place_list)

In [22]:
count("['foo', 'bah']")

2

In [20]:
df.head()

,div2_label,person,place,catnum
0,Miscellanies.,['Mr. Amyand.'],['East Indies'],['1799']
1,Miscellanies.,['Mr. Theobalds.'],['Norway'],['1801']
2,Miscellanies.,['Mr. Roberts'],[],['1806']
3,Miscellanies.,['Mr. M'],['Lisbon'],['1807']
4,Miscellanies.,[],['Lisbone'],['1808']


In [23]:
df_place_populated = df[df.place != '[]']
#  846 rows
df_place_populated['place_numbers'] = df_place_populated.apply(lambda x: fn(x['place']), axis=1)

/Users/deborahleem/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [24]:
df_place_populated.place_numbers.sum()

970

In [25]:
df_place_populated

,div2_label,person,place,catnum,place_numbers
0,Miscellanies.,['Mr. Amyand.'],['East Indies'],['1799'],1
1,Miscellanies.,['Mr. Theobalds.'],['Norway'],['1801'],1
3,Miscellanies.,['Mr. M'],['Lisbon'],['1807'],1
4,Miscellanies.,[],['Lisbone'],['1808'],1
5,Miscellanies.,['Dr. Stuart'],['Malacca'],['1809'],1
6,Miscellanies.,[],['China'],['1810'],1
7,Miscellanies.,['Mr. Mc. Cormick'],"['Lisbon', 'Brasile']",['1812'],2
8,Miscellanies.,['Mr. Hodsun'],['China'],['1813'],1
9,Miscellanies.,['Mr. Launce'],['Guinea'],['1816'],1
10,Miscellanies.,[],['Malabar'],['1818'],1


In [26]:
df_place_populated.place

0                              ['East Indies']
1                                   ['Norway']
3                                   ['Lisbon']
4                                  ['Lisbone']
5                                  ['Malacca']
6                                    ['China']
7                        ['Lisbon', 'Brasile']
8                                    ['China']
9                                   ['Guinea']
10                                 ['Malabar']
11                                 ['England']
12                                  ['London']
13                              ['Coromandel']
14                                   ['China']
15                                 ['Persian']
16                                 ['Tunquin']
17                        ['China', 'Suratte']
18                             ['New England']
19      ['Berbices', 'Surinam', 'West Indies']
20                                 ['Surinam']
21                                  ['Guinea']
22           